# **Big Data Analysis**

## **Pré-tratamento**

***Nesta secção irá ser feito o pré-tratamento dos dados***. Concretamente iremos ***fundir os dados*** provenientes dos diferentes datasets, *"alinhando-os"* pelas colunas *"Entidade"* (equivalente a nome do país), *"Code"* (equivalente ao código identificador IUPAC do país) e *"Year"* (correspondente ao ano em que foram recolhidos os dados).

Iremos também *"aparar"* os dados em função do indicador limitante, neste caso o ano de dados disponíveis, sendo o ***período de 1990 a 2016*** o período comum a todos os datasets.

**Para efeitos de comparação** iremos fazer merge dos datasets usando o package Pandas e PySpark, de modo a comparar a eficiência de ambos em diferentes contextos

### Pré-tratamento utilizando Pandas

In [ ]:
# Com pandas
import pandas as pd

# Importamos o módulo time para efeitos de comparação de tempos de execução
import time

#pd_start = time.time() # inicio do contador pandas

# Carregamos o dataset com dados populacionais
import pandas as pd
pop = pd.read_csv('datasets/population.csv')
pop.rename(columns={'Entity': 'Country'}, inplace = True)
pop = pop[pop['Year'] >= 1990]
pop = pop[pop['Year'] <= 2016]
pop.head()


,Country,Code,Year,Population (historical estimates)
227,Afghanistan,AFG,1990,10694804
228,Afghanistan,AFG,1991,10745168
229,Afghanistan,AFG,1992,12057436
230,Afghanistan,AFG,1993,14003764
231,Afghanistan,AFG,1994,15455560


In [ ]:
# Carregamos o dataset para prevelência de obesidade
obes = pd.read_csv('datasets/share-of-adults-defined-as-obese.csv')
obes.rename(columns={'Entity': 'Country'}, inplace = True)
obes = obes[obes['Year'] >= 1990]
obes.head()


,Country,Code,Year,"Indicator:Prevalence of obesity among adults, BMI &GreaterEqual; 30 (crude estimate) (%) - Sex:Both sexes"
15,Afghanistan,AFG,1990,1.0
16,Afghanistan,AFG,1991,1.1
17,Afghanistan,AFG,1992,1.2
18,Afghanistan,AFG,1993,1.2
19,Afghanistan,AFG,1994,1.3


In [ ]:
# Carregamos o dataset para prevalência doenças mentais
mental = pd.read_csv('datasets/mental-illnesses-prevalence.csv')
mental.rename(columns={'Entity': 'Country'}, inplace = True)
mental = mental[mental['Year'] <= 2016]
mental.head()


,Country,Code,Year,Schizophrenia disorders (share of population) - Sex: Both - Age: Age-standardized,Depressive disorders (share of population) - Sex: Both - Age: Age-standardized,Anxiety disorders (share of population) - Sex: Both - Age: Age-standardized,Bipolar disorders (share of population) - Sex: Both - Age: Age-standardized,Eating disorders (share of population) - Sex: Both - Age: Age-standardized
0,Afghanistan,AFG,1990,0.223206,4.996118,4.713314,0.703023,0.127700
1,Afghanistan,AFG,1991,0.222454,4.989290,4.702100,0.702069,0.123256
2,Afghanistan,AFG,1992,0.221751,4.981346,4.683743,0.700792,0.118844
3,Afghanistan,AFG,1993,0.220987,4.976958,4.673549,0.700087,0.115089
4,Afghanistan,AFG,1994,0.220183,4.977782,4.670810,0.699898,0.111815


In [ ]:
# Carregamos os dados relativos à pobreza (threshold abaixo de 6.85$ / dia)
# TODO para portefólio: adicionar mais elementos à pipeline como ver as dimensões, min maxing, etc.

poverty = pd.read_csv('datasets/poverty.csv')
poverty.rename(columns={'country': 'Country', 'year' : 'Year'}, inplace = True)
poverty = poverty[poverty['Year'] <= 2016]
poverty = poverty[poverty['Year'] >= 1990]
pov_filter = poverty[['Country','Year','headcount_ratio_international_povline','headcount_ratio_lower_mid_income_povline','headcount_ratio_upper_mid_income_povline']]
pov_filter.head()

,Country,Year,headcount_ratio_international_povline,headcount_ratio_lower_mid_income_povline,headcount_ratio_upper_mid_income_povline
0,Albania,1996,0.920669,11.174149,44.618417
1,Albania,2002,1.570843,14.132118,49.669635
2,Albania,2005,0.860527,8.715685,38.545254
3,Albania,2008,0.313650,5.250542,31.110345
4,Albania,2012,0.849754,6.182414,34.528906


In [ ]:
merged = pd.DataFrame()
merged = pd.merge(pop,obes,on=['Country','Code','Year'])
merged = pd.merge(merged,mental,on=['Country','Code','Year'])
merged = pd.merge(merged,pov_filter, left_on=['Country', 'Year'], right_on=['Country','Year'] )
pd_end = time.time() # fim do contador pandas

#pd_elapsed = pd_end - pd_start # tempo de execução do contador pandas


# Salvar o ficheiro com os datasets juntos na diretoria indicada
merged.to_csv('datasets/dataset_final.csv', index=False)

merged.head()

,Country,Code,Year,Population (historical estimates),"Indicator:Prevalence of obesity among adults, BMI &GreaterEqual; 30 (crude estimate) (%) - Sex:Both sexes",Schizophrenia disorders (share of population) - Sex: Both - Age: Age-standardized,Depressive disorders (share of population) - Sex: Both - Age: Age-standardized,Anxiety disorders (share of population) - Sex: Both - Age: Age-standardized,Bipolar disorders (share of population) - Sex: Both - Age: Age-standardized,Eating disorders (share of population) - Sex: Both - Age: Age-standardized,headcount_ratio_international_povline,headcount_ratio_lower_mid_income_povline,headcount_ratio_upper_mid_income_povline
0,Albania,ALB,1996,3271336,11.0,0.280476,2.401929,3.687558,0.541907,0.098910,0.920669,11.174149,44.618417
1,Albania,ALB,1996,3271336,11.0,0.280476,2.401929,3.687558,0.541907,0.098910,0.534846,8.898443,47.879143
2,Albania,ALB,2002,3123554,13.9,0.281883,2.461524,3.713175,0.542581,0.105739,1.570843,14.132118,49.669635
3,Albania,ALB,2002,3123554,13.9,0.281883,2.461524,3.713175,0.542581,0.105739,1.092647,11.388324,52.090966
4,Albania,ALB,2005,3032636,15.6,0.282246,2.472493,3.719762,0.542578,0.112578,0.860527,8.715685,38.545254


### Pré-tratamento de dados utilizando Spark

In [ ]:
#pip install pyspark
# remover # para instalar o package pyspark antes dos imports

from pyspark.sql import SparkSession
from pyspark.sql.functions import col

import time

# Criação de uma sessão Spark
Spark = SparkSession.builder.appName("BigDataAnalysis").getOrCreate()

#o tempo está a contar desde o inicio da execução da tarefa e não antes do início da sessão, mesmo assim Pysparks revelou ser mais lento neste caso
spark_start = time.time()

# Carregar e aparar datasets

# Dataset da população
spark_pop = Spark.read.csv(
    'datasets/population.csv',
    header = True ,
    inferSchema = True
    )

spark_pop = spark_pop.filter(col('Year').between(1990, 2016))

# Carregamos o dataset para prevelência de obesidade
spark_obes = Spark.read.csv(
    'datasets/share-of-adults-defined-as-obese.csv',
    header = True,
    inferSchema = True
    )

spark_obes = spark_obes.filter(col('Year') >= 1990)

# Carregamos o dataset para prevalência doenças mentais
spark_mental = Spark.read.csv(
    'datasets/mental-illnesses-prevalence.csv',
    header = True,
    inferSchema = True
    )

spark_mental = spark_mental.filter(col('Year') <= 2016)

# Unimos os datasets com o mesmo critério (país, IUPAC, código)

spark_merged = spark_pop.join \
  (spark_obes, on = ['Entity','Code','Year'], how = 'inner').join \
  (spark_mental, on = ['Entity', 'Code','Year'], how = 'inner')

# Descomentar para ver o dataset
#spark_merged.show()

Spark.stop()

spark_end = time.time() # fim do contador spark

spark_elapsed = spark_end - spark_start # tempo de execução do contador spark

In [ ]:
# spark_merged.write.format("csv").option("header", "true").mode("append").save('datasets/merged_spark.csv')
# não estou a conseguir salvar o ficheiro tratado com pyspark dá um erro estranho, relacionado com a diretoria,
# ja tentei com "append" e "overwrite", deve ser mesmo da diretoria

In [ ]:
# Comparamos o tempo de computação para cada um dos modelos

print(f'Com pandas, demorou {round(pd_elapsed, 3)} segundos',
      f'Com spark, demorou {round(spark_elapsed, 3)} segundos',
      sep = '\n')

### Análise do pré-tratamento
Neste caso concreto percebemos que várias ferramentas têm comportamentos diferentes em função dos dados, o que é algo a ter em conta no contexto do que pretendemos fazer.

Numa próxima abordagem iremos perceber que em função do que podemos fazer há ferramentas/abordagens mais ou menos eficientes. Neste caso concreto, a utilização de Spark demonstrou-se como menos eficiente.